# Estymacja cen opcji europejskich i barierowych metodą Monte Carlo
## Wprowadzenie do symulacji i metod Monte Carlo
### Mikołaj Płóciniczak, 2026

In [5]:
from pathlib import Path

from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
from IPython.display import HTML

ROOT = Path.cwd().resolve()
while not (ROOT / "parameters.py").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent

def show_code(path):
    code = (ROOT / path).read_text()
    formatter = HtmlFormatter(style="native")  # works well in dark mode
    styles = formatter.get_style_defs('.highlight')
    html = f"<style>{styles}</style>" + highlight(code, PythonLexer(), formatter)
    return HTML(html)


# 1. Wprowadzenie

Celem tego raportu jest estymacja cen opcji europejskiej oraz dyskretnej opcji barierowej typu up-and-out metodą Monte Carlo. Rozważamy proces ceny akcji opisany geometrycznym ruchem Browna (GBM) o postaci
$$
S(t) = S(0)\exp(\mu^* t + \sigma B(t)),
$$
gdzie $B(t)$ jest ruchem Browna, a parametry modelu są ustalone: $r = 0{,}05$, $\sigma = 0{,}125$, $\mu^* = r - \sigma^2/2 = -0{,}0125$, $S(0) = 100$ oraz $K = 100$.

W projekcie porównujemy kilka estymatorów Monte Carlo:
- klasyczny estymator typu Crude Monte Carlo,
- estymator ze stratyfikacją (dla podziału proporcjonalnego i optymalnego),
- estymator z antytetycznymi zmiennymi (dla $n = 1$),
- estymator z wykorzystaniem zmiennej kontrolnej (dla $n = 1$).

Dla przypadku opcji europejskiej ($C = \infty$) wyniki symulacji porównujemy z wartością dokładną daną wzorem Blacka–Scholesa. Dla skończonych wartości bariery $C$ analizujemy wpływ bariery na wartość opcji oraz efektywność poszczególnych technik redukcji wariancji.

# Metody symulacyjne

W tej części opisujemy sposób generowania ścieżek procesu GBM oraz konstrukcję różnych estymatorów Monte Carlo użytych w projekcie.

Poniżej przedstawiono krótkie omówienie modeli oraz metod, na których opiera się dalsza analiza.

## 2.1. Model geometrycznego ruchu Browna

Model geometrycznego ruchu Browna (GBM) opisuje ewolucję ceny aktywa jako
$$
S(t) = S(0)\exp(\mu^* t + \sigma B(t)),
$$
gdzie ruch Browna $B(t)$ stanowi źródło losowości. W projekcie ścieżki GBM powstają na podstawie wygenerowanego wektora wartości ruchu Browna o odpowiedniej kowariancji.  
Dla metod redukcji wariancji, takich jak stratyfikacja, stosujemy zmodyfikowane procedury generowania BM.

In [6]:
show_code("models/simulate_gbm_paths.py")

## 2.2. Definicja wypłaty opcji

W projekcie rozważamy dwie wypłaty: europejską oraz dyskretną barierową typu up-and-out.
Dla bariery $C < \infty$ wypłata wynosi zero, jeśli ścieżka ceny choć raz przekroczy poziom $C$; w przeciwnym razie jest to zdyskontowana wartość $(S(1) - K)_+$.  
Gdy $C = \infty$, otrzymujemy zwykłą opcję europejską.

In [7]:
show_code("payoffs/payoff_up_and_out_call.py")

## 2.3. Estymator Crude Monte Carlo

Podstawowy estymator Monte Carlo polega na generowaniu wielu niezależnych ścieżek GBM, obliczeniu wypłaty dla każdej z nich, a następnie uśrednieniu wyników. Jest to metoda referencyjna, do której porównujemy bardziej zaawansowane techniki redukcji wariancji.

In [8]:
show_code("methods/crude_monte_carlo.py")

## 2.4. Estymator stratyfikowany

W tym projekcie estymator stratyfikowany polega na estymacji wartości $I_{n,C}$ poprzez niezależne symulowanie ścieżek GBM warunkowo na przynależność odpowiadającego im wektora ruchu Browna do zadanych stratum. Wynik końcowy otrzymywany jest jako ważona średnia estymatorów cząstkowych z poszczególnych stratum, co pozwala na kontrolę rozkładu symulowanych trajektorii i redukcję wariancji względem crude Monte Carlo.

W dalszej części pracy rozważane będą dwie wersje estymatora stratyfikowanego, różniące się sposobem alokacji liczby symulacji pomiędzy poszczególne straty.

### 2.4.1. Rozmiar próby proporcjonalny

W przypadku podziału próby proporcjonalnego liczba symulacji w każdej stracie jest proporcjonalna do jej prawdopodobieństwa. Jest to naturalne rozszerzenie estymatora crude Monte Carlo, które zapewnia nieobciążoność estymatora, jednak nie wykorzystuje informacji o zróżnicowaniu wariancji wypłaty pomiędzy poszczególnymi stratum.

### 2.4.2 Rozmiar próby optymalny

## 2.5. Estymator antytetyczny

W tym projekcie estymator antytetyczny stosowany jest w przypadku $n=1$ i polega na generowaniu par zmiennych normalnych $(Z,−Z)$ do symulacji ruchu Browna w chwili $t=1$. Odpowiadające im wypłaty opcji są następnie uśredniane, co pozwala na redukcję wariancji estymatora $I_{1,C}$ względem crude Monte Carlo przy zachowaniu nieobciążoności.

In [11]:
show_code("methods/antithetic_monte_carlo.py")

## 2.6. Estymator zmiennej kontrolnej

W tym projekcie estymator z zmienną kontrolną stosowany jest w przypadku $n=1$; zgodnie z propozycją z treści zadania jako zmienną kontrolną przyjęto $X=B(1)$. Wykorzystanie znanej wartości oczekiwanej tej zmiennej pozwala skorygować estymator wypłaty opcji i zredukować wariancję estymatora $I_{1,C}$ względem crude Monte Carlo, przy zachowaniu nieobciążoności.

In [10]:
show_code("methods/control_variate.py")

# 3. Black-Scholes Benchmark

W przypadku $n = 1$ i $C = \infty$ rozważany instrument redukuje się do europejskiej opcji call, dla której znana jest wartość dokładna. W projekcie służy ona jako punkt odniesienia do porównania jakości estymatorów Monte Carlo z wynikami analitycznymi. Wartość ta dana jest przez formułę Blacka–Scholesa:
$$
I_{1,\infty} = \mathbb{E}\!\left[e^{-r}(S(1)-K)^+\right]
            = S(0)\,\Phi(d_1) - K e^{-r}\,\Phi(d_2),
$$
gdzie
$$
d_1 = \frac{1}{\sigma}\!\left(\log\frac{S(0)}{K} + r + \frac{\sigma^2}{2}\right),
\qquad
d_2 = d_1 - \sigma,
$$
a $\Phi$ oznacza dystrybuantę standardowego rozkładu normalnego.

In [9]:
show_code("models/black_scholes.py")

# 4. Ustawienia eksperymentów

# 5. Wyniki

# 6. Dyskusja

# 7. Wnioski końcowe